In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.feature import hog
from skimage.feature import local_binary_pattern
import os
from FeatureExtractor import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier


In [2]:
Videos = ['Pigs_49651_960_540_500f','Koi_5652_952_540',\
          'Pigeons_8234_1280_720','Pigeons_4927_960_540_600f', \
          'Pigeons_29033_960_540_300f']

# The individual clips should be stored in folders with the respective
# names. For example, folder 'Koi_5652_952_540_clips' should contain
# subfolders 'Catherine', 'Dwayne', 'Florence', etc, with the 
# individual images. MATLAB code for splitting the video, storing the
# frames and creating the folders with the clips is provided in this
# repository.

cl_names = ['Linear Discriminant Analysis', '3-nn', 'Decision Tree',
            'SVM', 'Bagging', 'Random Forest'] # classifiers

In [3]:
#---------------------------------------------------------
def select_half_video(folder,bb_file,part = 1):
    
    bb = pd.read_csv(bb_file,header = None)    
    bb = bb.to_numpy()
   
    # Find the number of frames in the video
    lasf_file_name = bb[-1,5]
    f = np.int(lasf_file_name[5:10]) 
    half_f = np.floor(f/2).astype(int)
    
    images, labels = [],[]
    for i in range(len(bb)):
        z = bb[i,5] # take the file name
        nn = np.int(z[5:10]) # convert to numeric
        flag = ((part == 1) & (nn <= half_f)) | \
            ((part == 2) & (nn > half_f)) 
        if flag:
            label = bb[i,0]# take the string label
            label = label.replace(' ','') # trim the blanks
            filename = bb[i,5] 
            fn = label+'/'+ label + '_frame_' + filename[5:10] + '.jpg'
            img = plt.imread(folder+'/'+fn)
            if img is not None:
                images.append(img)
                labels.append(label)
            else:
                print(folder+'/'+fn)     
            
    return images, labels
        

In [4]:
def training_testing_rows(bb_file,part = 1):
    
    bb = pd.read_csv(bb_file,header = None)    
    bb = bb.to_numpy()
   
    # Find the number of frames in the video
    lasf_file_name = bb[-1,5]
    f = np.int(lasf_file_name[5:10]) 
    half_f = np.floor(f/2).astype(int)
    
    index = []
    for i in range(len(bb)):
        z = bb[i,5] # take the file name
        nn = np.int(z[5:10]) # convert to numeric
        flag = ((part == 1) & (nn <= half_f)) | \
            ((part == 2) & (nn > half_f)) 
        if flag:
            index.append(i)
    return index

In [5]:
def extract_features(images, r = 50, verbose = False):
    # Extracts basic features.
    # All images are first resized to r-by-r pixels.
    # Use verbose = True to see a 4x4 grid of images 
    
    if verbose:
        plt.figure(figsize = (10,10))
        k = 1 # subplot index
        khog = 1
        grid_size = 4
    
    for img in images:
        if verbose:
            if k <= grid_size**2:
                plt.subplot(grid_size,grid_size,k)
                plt.imshow(img)
                plt.axis('Off')
                k +=1   
        
        resized_img = resize(img, (r, r))
        
        #creating HOG features ----------------------------------------------------
        fd = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), \
                    cells_per_block=(1, 1), multichannel=True)

        fd = np.reshape(fd,(1,-1))
        if 'DataHOG' in locals():
            DataHOG = np.append(DataHOG,fd,axis = 0)
        else:
            DataHOG = fd

        radius = 3
        n_points = 8 * radius
        method = 'default'
        
        #creating LBP features ----------------------------------------------------
        nbins = 50 # this will be my number of features
        
        lbp_raw = local_binary_pattern(rgb2gray(img), n_points, radius, method)
        lbp,_ = np.histogram(lbp_raw,bins = 50)
        lbp = lbp/np.sum(lbp)
        lbp = np.reshape(lbp,(1,-1))

        if 'DataLBP' in locals():
            DataLBP = np.append(DataLBP,lbp,axis = 0)
        else:
            DataLBP = lbp
            
        #creating Block RGB features -----------------------------------------------       
        rgb = fox_get_colour_features(img, fstr = 'RGB', blocks_r = 3, 
            blocks_c = 3, bins = 20)
        rgb = np.reshape(rgb,(1,-1))
        if 'DataRGB' in locals():
            DataRGB = np.append(DataRGB,rgb,axis = 0)
        else:
            DataRGB = rgb
            
        #creating H10 features ---------------------------------------------------
        hsv_img = rgb2hsv(img)
        hue_img = hsv_img[:, :, 0]
        
        h10 = fox_get_colour_features(img, fstr = 'H', blocks_r = 1, 
            blocks_c = 1, bins = 10)
        
        h10 = np.reshape(h10,(1,-1))
        if 'DataH10' in locals():
            DataH10 = np.append(DataH10,h10,axis = 0)
        else:
            DataH10 = h10
        
    if verbose:
        plt.show()
   
    return DataHOG, DataLBP, DataRGB, DataH10

In [6]:
##############################################
# CALCULATIONS

# Calculate and display the training and testing accuracies
# of the 6 classifiers and the 4 data representations:
# HOG, LBP, RGB, H10. The data representations as well as
# the labels for the training and testing data are stored 
# in csv files. Each file contains the training data
# followed by the testing data.

Sizes = np.zeros((len(Videos),3)) 
# info about training and testing data

plot_flag = False
r = 56

for v in range(len(Videos)):
    
    video = Videos[v]

    print('\n\n#####  ' + video + '  #####\n\n')
    
    folder = "../" + video + "_clips"
    bb_file = "../BB_" + video + ".csv"

    # Read training and testing data (half video; must not be random!)
    imds_1,labels_1 = select_half_video(folder,bb_file,part = 1) # training
    imds_2,labels_2 = select_half_video(folder,bb_file,part = 2) # testing
    
    # Repair the data (missing classes in training data)
    # Find missing classes and remove those from both sets

    ll = list(np.sort(np.unique(labels_1)))
    n_classes = len(ll)

    X_train_im = []
    X_train = []
    y_train_num = []
    for i in range(len(imds_1)):
        y_train_num.append(ll.index(labels_1[i]))
        X_train.append(resize(imds_1[i], (r, r)))
        X_train_im.append(imds_1[i])   
    X_train = np.array(X_train)/255                       

    X_test_im = []
    X_test = []
    y_test_num = [] # numerical testing labels
    for i in range(len(imds_2)):
        if labels_2[i] in set(ll):
            y_test_num.append(ll.index(labels_2[i]))
            X_test.append(resize(imds_2[i], (r, r)))
            X_test_im.append(imds_2[i])
    X_test = np.array(X_test)/255    

    print('Training size = ',X_train.shape,' Testing size = ',X_test.shape, '\n')
    
    Sizes[v,0] = len(y_train_num)
    Sizes[v,1] = len(y_test_num)
    Sizes[v,2] = n_classes
    
    print(Sizes)
    
    data_train = extract_features(X_train, r = r, verbose = False)
    data_test = extract_features(X_test, r = r, verbose = False)
    
    # Plot some classes to verify that the labels are intact
    if plot_flag: 
        
        zzz = np.where(np.array(y_train_num) == 2)
        Class_train_2 = []
        for k in range(len(zzz[0])):
            p = X_train_im[zzz[0][k]]
            Class_train_2.append(p)

        zzz = np.where(np.array(y_test_num) == 3)
        Class_test_3 = []
        for k in range(len(zzz[0])):
            p = X_test_im[zzz[0][k]]
            Class_test_3.append(p)

        plt.figure()
        print(ll[2])
        random_image_montage(Class_train_2)

        plt.figure()
        print(ll[3])
        random_image_montage(Class_test_3)

    # Classification --------------------------------
        
    cla = [] # accumlate the classifiers
    cla.append(LinearDiscriminantAnalysis())
    cla.append(KNeighborsClassifier())
    cla.append(DecisionTreeClassifier())
    cla.append(SVC(gamma=0.1, kernel="rbf"))
    cla.append(BaggingClassifier())
    cla.append(RandomForestClassifier())
    
    DataNames = ['DataHOG', 'DataLBP', 'DataRGB', 'DataH10']
    

    accs = np.zeros((len(cl_names),len(DataNames)))
    accs_tr = np.zeros((len(cl_names),len(DataNames)))
    for i in range(len(cl_names)):
        classifier = cla[i]
        for j in range(len(DataNames)):
            current_training_data = data_train[j]
            current_testing_data = data_test[j]
            classifier.fit(current_training_data,y_train_num)
            assigned_labels = classifier.predict(current_testing_data)
            accs[i,j] = np.mean(y_test_num == assigned_labels)
            assigned_labels = classifier.predict(current_training_data)
            accs_tr[i,j] = np.mean(y_train_num == assigned_labels)

    print('Training\n',accs_tr)
    print('\nTesting\n',accs)

    # Save results
    np.savetxt("ClassifierAccSimpleFeatures_"+ \
        video+".csv", accs, delimiter=",")   
    
    DataHOG = np.vstack((data_train[0],data_test[0]))
    DataLBP = np.vstack((data_train[1],data_test[1]))
    DataRGB = np.vstack((data_train[2],data_test[2]))
    DataH10 = np.vstack((data_train[3],data_test[3]))
    Labels = np.vstack((np.reshape(y_train_num,(-1,1)), \
                        np.reshape(y_test_num,(-1,1))))
    
    
    np.savetxt(video+"_DataHOG.csv", DataHOG, delimiter=",")
    np.savetxt(video+"_DataLBP.csv", DataLBP, delimiter=",")
    np.savetxt(video+"_DataRGB.csv", DataRGB, delimiter=",")
    np.savetxt(video+"_DataH10.csv", DataH10, delimiter=",")
    np.savetxt(video+"_Labels.csv", Labels, delimiter=",")

print(Sizes)
np.savetxt('TrainingTestingSizes.csv', Sizes, delimiter=",")
        




#####  Pigs_49651_960_540_500f  #####


Training size =  (2711, 56, 56, 3)  Testing size =  (3212, 56, 56, 3) 

[[2711. 3212.   22.]
 [   0.    0.    0.]
 [   0.    0.    0.]
 [   0.    0.    0.]
 [   0.    0.    0.]]
Training
 [[0.97196606 0.6370343  0.94835854 0.50202877]
 [0.97786795 0.85171523 0.98081889 0.73921062]
 [1.         1.         1.         0.9988934 ]
 [0.99668019 0.08926595 0.08926595 0.22168941]
 [0.99631132 0.99557359 0.99926226 0.98487643]
 [1.         1.         1.         0.9988934 ]]

Testing
 [[0.22882939 0.17372354 0.36830635 0.21699875]
 [0.22540473 0.1369863  0.27303861 0.19178082]
 [0.10990037 0.10305106 0.1858655  0.18337484]
 [0.23599004 0.04763387 0.04763387 0.14632628]
 [0.15535492 0.13013699 0.2135741  0.19769614]
 [0.2107721  0.16002491 0.28891656 0.20765878]]


#####  Koi_5652_952_540  #####


Training size =  (916, 56, 56, 3)  Testing size =  (719, 56, 56, 3) 

[[2711. 3212.   22.]
 [ 916.  719.    9.]
 [   0.    0.    0.]
 [   0.    0.    0.]
 [   